In [18]:
!pip install -q langgraph langchain langchain-groq python-dotenv

In [19]:
import os
import random
from typing import Annotated, TypedDict, Literal

from dotenv import load_dotenv

from langchain_core.tools import tool
from langchain_groq import ChatGroq

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [20]:
load_dotenv()  # loads .env from current folder

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    groq_api_key=GROQ_API_KEY,
    temperature=0
)

print("Groq LLM initialized")

Groq LLM initialized


In [21]:
@tool
def flip_coin():
    """Returns either 'Heads' or 'Tails' randomly."""
    return f"The coin landed on: {random.choice(['Heads', 'Tails'])}"

@tool
def roll_dice():
    """Returns a random number between 1 and 6."""
    return f"The die rolled a: {random.randint(1, 6)}"

print("Tools defined")

Tools defined


In [22]:
llm_with_tools = llm.bind_tools([flip_coin, roll_dice])

print("Tools bound to LLM")

Tools bound to LLM


In [26]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [27]:
def chatbot_node(state: State):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

In [28]:
def tool_executor_node(state: State):
    last_message = state["messages"][-1]
    tool_messages = []

    for tool_call in last_message.tool_calls:
        name = tool_call["name"]
        args = tool_call.get("args", {})

        if name == "flip_coin":
            result = flip_coin.invoke(args)
        elif name == "roll_dice":
            result = roll_dice.invoke(args)
        else:
            result = "Unknown tool"

        tool_messages.append(result)

    return {"messages": tool_messages}

In [30]:
def should_continue(state: State) -> Literal["tools", END]:
    last_message = state["messages"][-1]

    if last_message.tool_calls:
        return "tools"

    return END

In [31]:
builder = StateGraph(State)

builder.add_node("chatbot", chatbot_node)
builder.add_node("tools", tool_executor_node)

builder.add_edge(START, "chatbot")

builder.add_conditional_edges(
    "chatbot",
    should_continue,
    {"tools": "tools", END: END}
)

# ❌ NO edge from tools → chatbot (prevents infinite loop)

graph = builder.compile()

print("LangGraph 1.0 agent compiled successfully!")

LangGraph 1.0 agent compiled successfully!


In [34]:
result = graph.invoke({
    "messages": [("user", "Flip a coin")]
})

print(result["messages"][-1].content)

The coin landed on: Tails


In [35]:
result = graph.invoke({
    "messages": [("user", "Roll a dice")]
})

print(result["messages"][-1].content)

The die rolled a: 5
